# Exercise 3: EIOA: basic theory & calculations

### Objectives
- Understand the format of input-output tables
- Interpret the Leontief inverse
- Calculate the carbon footprint

## Exercise

A fictitious nation has a very basic national economy consisting of five sectors, each producing a single type of goods for sale: rice, beef, electricity, car, and insurance. The following data were obtained from the nation's Bureau of Statistics:
Inter-industry transactions and value added (unit: €/year):

In [2]:
# Firt import numpy and pandas
import numpy as np
import pandas as pd

#### Inter-industry transactions and value added (unit: €/year):

In [3]:
# Products are the rows, industry are the columns
Z = np.array([
    [100, 1900, 0, 0, 0],
    [0, 1000, 0, 0, 0],
    [1600, 2500, 900, 1000, 4000],
    [500, 1500, 1500, 1000, 1500],
    [300, 700, 3000, 2000, 1000]
], dtype=np.int32)
Z

Z = pd.DataFrame(
     Z,
     index=pd.Index(["rice", "beef", "electricity", "car", "insurance"], name="product"),
     columns=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry")
 )
Z

industry,rice farm,cattle farm,power plant,car maker,insurance company
product,,,,,
rice,100,1900,0,0,0
beef,0,1000,0,0,0
electricity,1600,2500,900,1000,4000
car,500,1500,1500,1000,1500
insurance,300,700,3000,2000,1000


#### The value of finished products sold (final consumption,€/year):

In [22]:
# Value added (last row of the first table)
v = np.array([2500, 2400, 9600, 16000, 8500], dtype=np.int32)
v = pd.DataFrame(v, columns=["Value added"], index=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry")).T
v

industry,rice farm,cattle farm,power plant,car maker,insurance company
Value added,2500,2400,9600,16000,8500


#### Final value of the products sold
KEEP IN MIND THE SHAPE!

In [23]:
y = np.array([3000, 9000, 5000, 14000, 8000], dtype=np.int32)
y = pd.DataFrame(y,  index=pd.Index(["rice", "beef", "electricity", "car", "insurance"], name="product"), columns=["Final Demand"])
y

,Final Demand
product,
rice,3000
beef,9000
electricity,5000
car,14000
insurance,8000


#### The direct water use and CO2 emissions by sector. (Note: this is the environmental extension matrix F)

In [15]:
F = np.array([
    [10000, 2000, 15000, 1000, 750],
    [50, 500, 7500, 300, 15]
], dtype=np.int32)

F = pd.DataFrame(F, 
                 index=pd.Index(["Water Use (L)", "CO2 Emissions (kg)"], name="extensions"), 
                 columns=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry"))
F


industry,rice farm,cattle farm,power plant,car maker,insurance company
extensions,,,,,
Water Use (L),10000,2000,15000,1000,750
CO2 Emissions (kg),50,500,7500,300,15


#### Total outputs: intermediate output + final product output

Multiply the products/industry table (Z) by shaped array of ones...
i = np.ones((5,1))  # shape (5, 1)
z_sum = np.dot(Z, i)  # shape (5, 1)

... Or just get the sum of the rows while keeping the dimensions.
Z_sum = Z.sum(axis=1)  # Wrong dimensions: shape (5, )

In [7]:
Z_sum = Z.sum(axis=1, keepdims=True)  # shape (5, 1)
x_out = Z_sum + y
x = x_out
x_out

array([[ 5000],
       [10000],
       [15000],
       [20000],
       [15000]])

In [8]:
# Total inputs: intermediate input + value added
Z_sum = Z.sum(axis=0, keepdims=True)  # shape (1, 5)
x_in = Z_sum + v
x_in

array([[ 5000, 10000, 15000, 20000, 15000]])

In [9]:
# Task 2 use of water, CO2 and wage payment.
# Use of water -> F water divided by total output
# Make sure to transpose x to ensure a (1, 5) shape
f_water = F_water / x.transpose()
f_water

array([[2.  , 0.2 , 1.  , 0.05, 0.05]])

In [10]:
# Emission of CO2 -> F CO2 divided by total output
# Same with transposing here.
f_co2 = F_co2 / x.transpose()
f_co2

array([[0.01 , 0.05 , 0.5  , 0.015, 0.001]])

In [11]:
# Wage payment ->  Value added divided by total output
# Transpose again.
f_va = v / x.transpose()
f_va

array([[0.5       , 0.24      , 0.64      , 0.8       , 0.56666667]])

In [12]:
# Or, if we want to have everything in a single array:
f_all = F / x.transpose()

# And append the wage payment row at the end
f_all = np.concatenate((f_all, f_va), axis=0)
f_all

array([[2.00000000e+00, 2.00000000e-01, 1.00000000e+00, 5.00000000e-02,
        5.00000000e-02],
       [1.00000000e-02, 5.00000000e-02, 5.00000000e-01, 1.50000000e-02,
        1.00000000e-03],
       [5.00000000e-01, 2.40000000e-01, 6.40000000e-01, 8.00000000e-01,
        5.66666667e-01]])

In [13]:
# Calculate the leontief inverse matrix

# Create the technical coefficient matrix A first
A = Z / x.transpose()
# Create an identity matrix the same shape as A
I = np.identity(5)
# Use numpy linear algebra inverse function to get L = (I - A)^-1
L = np.linalg.inv(I - A)
L

array([[1.02040816, 0.2154195 , 0.        , 0.        , 0.        ],
       [0.        , 1.11111111, 0.        , 0.        , 0.        ],
       [0.40461549, 0.44595385, 1.14586964, 0.09585221, 0.33766121],
       [0.1679281 , 0.26837237, 0.14813524, 1.07703032, 0.15772046],
       [0.17029328, 0.22149745, 0.26141513, 0.13593587, 1.16068316]])

In [14]:
# Task 4 Insurance's water multiplier
f_tot_water = f_water.dot(L)
f_tot_water

array([[2.46234288, 1.12350857, 1.16634716, 0.15650052, 0.40358139]])

In [15]:
# We can also store the information we have about all of the processes
f_tot_water_p = np.diagflat(f_water).dot(L)
f_tot_water_p

array([[2.04081633, 0.430839  , 0.        , 0.        , 0.        ],
       [0.        , 0.22222222, 0.        , 0.        , 0.        ],
       [0.40461549, 0.44595385, 1.14586964, 0.09585221, 0.33766121],
       [0.0083964 , 0.01341862, 0.00740676, 0.05385152, 0.00788602],
       [0.00851466, 0.01107487, 0.01307076, 0.00679679, 0.05803416]])

In [16]:
# And when we sum the columns, we find that we get the same result.
np.isclose(f_tot_water_p.sum(axis=0), f_tot_water)

array([[ True,  True,  True,  True,  True]])

In [17]:
# Compare f_total_water (direct + indirect) vs. f_water (direct)
print(f"Total water (direct + indirect):\n{f_tot_water}\n\nDirect water only:\n{f_water}")

Total water (direct + indirect):
[[2.46234288 1.12350857 1.16634716 0.15650052 0.40358139]]

Direct water only:
[[2.   0.2  1.   0.05 0.05]]


In [18]:
# Carbon footprint: We use the calculated co2 and perform a dot multiplication:
# First with the leontief inverse matrix and then with the value of the products sold.
EF_co2 = f_co2.dot(L).dot(y)
EF_co2

array([[8365.]])

In [19]:
# We can also just combine all of the footprints and calculate the same.
EF = f_all.dot(L).dot(y)
EF  # In order: Water, CO2 and value added by final demand

array([[28750.],
       [ 8365.],
       [39000.]])

In [20]:
# Carbon footprint related to final demand:
EF_co2_exp = f_co2 @ L @ np.diagflat(y)
EF_co2_exp

array([[ 645.60312204, 2564.40384903, 2877.09132102,  899.0449634 ,
        1378.85674451]])

In [23]:
# Carbon footprint of traced to producing sector
EF_co2_p = np.diagflat(f_co2) @ L @ y
EF_co2_p

array([[  50.],
       [ 500.],
       [7500.],
       [ 300.],
       [  15.]])

In [24]:
# Combining the two calculation above, we have all of the information in one place
EF_co2_exp_p = np.diagflat(f_co2) @ L @ np.diagflat(y)
EF_co2_exp_p

array([[3.06122449e+01, 1.93877551e+01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 5.00000000e+02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [6.06923233e+02, 2.00679235e+03, 2.86467410e+03, 6.70965493e+02,
        1.35064482e+03],
       [7.55676443e+00, 3.62302694e+01, 1.11101429e+01, 2.26176368e+02,
        1.89264552e+01],
       [5.10879849e-01, 1.99347707e+00, 1.30707564e+00, 1.90310213e+00,
        9.28546532e+00]])

In [25]:
# Lets check if everything comes up correctly:
assert EF_co2 == EF_co2_p.sum()
assert EF_co2 == EF_co2_exp.sum()
assert EF_co2 == EF_co2_exp_p.sum()
# If no errors show, all of the matrixes sum to the same value as EF_co2

In [26]:
# Lets also check that the EF_co2_exp_p can sum to the same values as its parts
# In this case we use 'isclose' because comparing floats is difficult to do exactly.
assert np.isclose(EF_co2_p, EF_co2_exp_p.sum(axis=1, keepdims=True)).all()
assert np.isclose(EF_co2_exp, EF_co2_exp_p.sum(axis=0, keepdims=True)).all()
# If no errors show, the EF_co2_exp_p matrix sums to the same/similar values as EF_co2_p and EF_co2_exp.

In [27]:
# Comparing float values doesn´t always work.
EF_co2_exp_p.sum(axis=1, keepdims=True) == EF_co2_p

array([[ True],
       [ True],
       [False],
       [ True],
       [False]])